In [4]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import sklearn
import os, pathlib, shutil
import kaggle, kagglehub


Download the competition


In [5]:
competition_name = "home-data-for-ml-course"  # Change this to the competition you want to download
competition_path = pathlib.Path(
    kagglehub.competition.competition_download(competition_name)
)  # Downloads the competition
competition_files: list = os.listdir(competition_path)  # List of the files included in the competition

print(competition_files)

# copy files to the current directory
shutil.copyfile(competition_path / "sample_submission.csv", "sample_submission.csv")
shutil.copyfile(competition_path / "test.csv", "test.csv")
shutil.copyfile(competition_path / "train.csv", "train.csv")

['data_description.txt', 'sample_submission.csv', 'sample_submission.csv.gz', 'test.csv', 'test.csv.gz', 'train.csv', 'train.csv.gz']


'train.csv'

In [42]:
pl.Config.set_tbl_rows(20)
train_df = pl.read_csv(competition_path / "train.csv", ignore_errors=True)
test_df = pl.read_csv(competition_path / "test.csv", ignore_errors=True)
sample_submission_df = pl.read_csv(competition_path / "sample_submission.csv")

target_feature_name: str = (set(train_df.columns) - set(test_df.columns)).pop()
target_col = train_df.get_column(target_feature_name)
print(train_df.drop(target_feature_name).schema == test_df.schema)
df = pl.concat([train_df, test_df.with_columns(pl.lit(-1).cast(pl.Int64).alias(target_feature_name))])

True


### EDA


In [92]:
numeric_cols_df = train_df.drop("Id", "SalePrice").select(pl.selectors.numeric())
pearson_correlations = (
    numeric_cols_df.select(pl.corr(pl.col("*"), target_col).abs())
    .transpose(include_header=True, header_name="Features")
    .rename({"column_0": "Correlation"})
    .sort("Correlation", descending=True)
)
pearson_correlations.head()

Features,Correlation
str,f64
"""OverallQual""",0.790982
"""GrLivArea""",0.708624
"""GarageCars""",0.640409
"""GarageArea""",0.623431
"""TotalBsmtSF""",0.613581


In [102]:
numeric_col_names_sorted = pearson_correlations.get_column("Features").to_numpy()
sorted_df = df.sort(by=["OverallQual", "GrLivArea"], descending=True)
numeric_cols_df = sorted_df.select("Id", *numeric_col_names_sorted, "SalePrice")
str_cols_df = sorted_df.select(pl.selectors.string()).cast(pl.Categorical)
sorted_df = numeric_cols_df.hstack(str_cols_df)
sorted_df.write_parquet("train_test_merged_df_bak.parquet", statistics="full")
